In [1]:
import matplotlib.pyplot as plt
%matplotlib notebook
plt.ioff()
from skimage import io
from skimage.draw import disk, circle_perimeter
import pandas as pd
import numpy as np
from os import path

In [2]:
base_path = "/media/ula/DATADRIVE1/fos_gfp_tmaze/ctx_landmark/despeckle/"
res_dir_path = base_path + "alignment_result/"
dir_path = res_dir_path + "aligned_despeckle/"
coords_path = res_dir_path + "aligned_despeckle/overlap_coords/"

cell_data_fn_template = "m{}r{}_{}_output.txt"
coords_fn_template = "m{}r{}_{}.txt"
img_fn_template = "m{}r{}_{}.tif"

In [3]:
sessions={
'l' : ["landmark", "ctx1", "ctx2"],
'c' : ["ctx", "landmark1", "landmark2"]}

In [4]:
def get_quantiles(df, columns, q):
    quantiles = np.array([])
    for col in columns:
        quantiles = np.append(quantiles, df[df[col]>0][col].quantile(q))
    return quantiles

In [5]:
def filter_by_quantiles(df, columns, q):
    qs = get_quantiles(df, columns, q)
    for qu, col in zip(qs, columns):
        df = df[df[col]>qu]
    return df

In [23]:
def merge_coord_dfs(mouse, region, session_ids, s_code, int_df):
    session_order = sessions[s_code]
    colnames = np.array([])
    for s_id in session_ids:
        df = pd.read_csv(dir_path + cell_data_fn_template
                     .format(mouse, region, session_order[s_id]), "\t", header=1)
        s_id = str(s_id)
        df = df.rename(columns={"Intensity center X (px) (ch 0)":"X"+s_id,
                               "Intensity center Y (px) (ch 0)":"Y"+s_id,
                               "Intensity center Z (px) (ch 0)":"Z"+s_id})
        int_df = int_df.merge(df, right_index=True, left_on="idx_s"+s_id)
        colnames = np.append(colnames,["X"+s_id, "Y"+s_id, "Z"+s_id])
    return int_df[colnames]

In [26]:
def create_ovlap_coords_file(mouse, region, s1_id, s2_id, s_code):
    session_order = sessions[s_code]
    
    ord_cells = pd.read_csv(dir_path + cell_data_fn_template.format(mouse,region, "merged"))
    ord_cells = filter_by_quantiles(ord_cells, ['i'+str(s1_id), 'i'+str(s2_id)], 0.25)
    ord_cells = merge_coord_dfs(mouse, region, [s1_id, s2_id], s_code, ord_cells)
    
    ord_cells.to_csv(coords_path+coords_fn_template.format(mouse, region, str(s1_id)+"_"
                                                           +str(s2_id)))
    
    
    

In [27]:
create_ovlap_coords_file(1,1,1,2,'l')